# Rendmi

Datadump used updated untl 06/07/2022

Source: http://dati.isprambiente.it/accesso-ai-dati/download/

In order to analyze tyhe dataset, gather all the data needed and turn it into a .json file we will use rdflib to query the graph.

In [11]:
# all imports
import pprint
import rdflib
from rdflib import URIRef, Literal, Namespace
from rdflib.namespace import XSD, RDFS, DCTERMS
from rdflib import Literal

Data had some errors that needed to be manualy corrected in order to maake the parser work.

**Specifically**:

Invalid lines sucH as *-schema#label>"[ sometext ]".*  were changed into *-schema#label> "[ sometext ]".* 

In [12]:
# create an empty Graph
g = rdflib.Graph()

# parse a local RDF file by specifying the format into the graph
result = g.parse("../datasets/originals/dissesto_07_06_2022.nt", format='nt')

Looks abit weird as 50 entries are missing w.r.t. the original .nt file

Next step consists in subsetting thedataset nd keeping oly facts relted to Emilia romagna.

Thus we previously gathered a [list](http://dati.isprambiente.it/sparql?default-graph-uri=&query=select+distinct+%3Fp+%3Fplabel%0D%0Awhere+%7B%0D%0A%3Fs+%3Fp+%3Fo.%0D%0A%3Fp+rdfs%3Alabel+%3Fplabel.%0D%0Afilter%28lang%28%3Fplabel%29+%3D+%27en%27%29%7D%0D%0A&format=text%2Fhtml&timeout=0&debug=on) of all predicates in the dataset and individuated http://www.geonames.org/ontology#parentADM1 as a startng point, as it is the predicate referring to the italian region 

In [23]:
#look at regions

types = set()

type = URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type') 
instability = '<http://dati.isprambiente.it/ontology/core#Instability>'

for s, p, o in result.triples((None, type, None)):
    types.add(o)

print(types)

{rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#EconomicIndicator'), rdflib.term.URIRef('http://purl.org/goodrelations/v1#BusinessEntity'), rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#Concept'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#LotStep'), rdflib.term.URIRef('http://purl.org/procurement/public-contracts#FrameworkAgreement'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#Contract'), rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#ConceptScheme'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#Lithology'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#Intervention'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#AuthorityKind'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#Instability'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#Repair')}


The result shows the main types of entities recorded in the graph.
At the moment the interesting fieldsfor us are Intervention, Instability and repair  interesting for us here. 

Lets see which predicateds are used with each one of them, keeping in mind weneed to find their geo loction, entity, etc. 

In [85]:
#Intervention

interventions = set()
type = URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type') 
intervention = URIRef('http://dati.isprambiente.it/ontology/core#Intervention') 

for s, p, o in result.triples((None, type, intervention)):
    interventions.add(s)

print(list(interventions)[0])

http://dati.isprambiente.it/id/ihi/intervention/232/02


Now that we have gathered all regords having intervention as type, lets see with a random one the predicates related to it.

In [86]:
#predicates
preds_interventions = set()
intervention = URIRef('http://dati.isprambiente.it/id/ihi/intervention/232/02') 

for s, p, o in result.triples((intervention, None, None)):
    preds_interventions.add(p)

print(preds_interventions)

{rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#amountFinanced'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#hasAgreement'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#contractingAuthority'), rdflib.term.URIRef('http://purl.org/dc/terms/isPartOf'), rdflib.term.URIRef('http://purl.org/dc/terms/publisher'), rdflib.term.URIRef('http://www.w3.org/2003/01/geo/wgs84_pos#location'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#primaryGeographicalFeature'), rdflib.term.URIRef('http://purl.org/dc/terms/isReferencedBy'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#officialInstabilityType'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')}


In [87]:
#test

for intervention_uri in interventions:
    intervention_this = URIRef(intervention_uri) 
    pred_this = set()

    for s, p, o in result.triples((intervention, None, None)):
        pred_this.add(p)
    
    if pred_this != preds_interventions:
        print(intervention_this, 'has different predicates')

In [88]:
#Instability

instabilities = set()
type = URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type') 
instability = URIRef('http://dati.isprambiente.it/ontology/core#Instability') 

for s, p, o in result.triples((None, type, instability)):
    instabilities.add(s)

print(list(instabilities)[0])

http://dati.isprambiente.it/id/ihi/instability/108/082di-6


In [89]:
# predicates
preds_instability = set()
instability = URIRef('http://dati.isprambiente.it/id/ihi/instability/108/082di-6') 

for s, p, o in result.triples((instability, None, None)):
    preds_instability.add(p)

print(preds_instability)

{rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#instabilityGroup'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#instabilityRelatedTo'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#instabilityType')}


In [90]:
#test

for intervention_uri in instabilities:
    instability_this = URIRef(intervention_uri) 
    preds_this = set()

    for s, p, o in result.triples((instability_this, None, None)):
        preds_this.add(p)
    
    if preds_this != preds_instability:
        print(instability_this, 'has different predicates')

In [95]:
#Repair

repairs = set()
type = URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type') 
repair = URIRef('http://dati.isprambiente.it/ontology/core#Repair') 

for s, p, o in result.triples((None, type, repair)):
    repairs.add(s)

print(list(repairs)[0])

http://dati.isprambiente.it/id/ihi/repair/036/061sf-32


In [97]:
# predicates
preds_repairs = set()
repair = URIRef('http://dati.isprambiente.it/id/ihi/repair/036/061sf-32') 

for s, p, o in result.triples((repair, None, None)):
    preds_repairs.add(p)

print(preds_repairs)

{rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#repairRelatedTo'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#repairType')}


In [112]:
#test
x= 0
for repair_uri in repairs:
    repair_this = URIRef(repair_uri) 
    preds_thiss = set()

    for s, p, o in result.triples((repair_this, None, None)):
        preds_thiss.add(p)
    
    if preds_thiss != preds_repairs:
        x +=1
        print(repair_this, 'has different predicates', preds_thiss)

print(x)

http://dati.isprambiente.it/id/ihi/repair/226/991sf-45 has different predicates {rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#repairType'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#repairRelatedTo'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#repairCategory')}
http://dati.isprambiente.it/id/ihi/repair/053/071sf-22 has different predicates {rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#repairType'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#repairRelatedTo'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#repairCategory')}
http://dati.isprambiente.it/id/ihi/repair/334/061sf-21 has different predicates {rdflib.te

In [107]:
print(len(repairs))

12600


**Test Results:**
4447 repair entries out of 12600 have an additional preposition, i.e., rdflib.term.URIRef('http://dati.isprambiente.it/ontology/core#repairCategory')}